In [1]:
import os

In [2]:
%pwd

'd:\\End-to-End-Chicken-Disease-Project-1\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-End-Chicken-Disease-Project-1'

In [5]:
# Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
# Import all configuration settings from constants and utils

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directory

In [7]:
# Configuration Manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
# import packages for Components

import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File Already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extract the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# Pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-11 10:54:53,184: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-11 10:54:53,194: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-11 10:54:53,200: INFO: common: created directory at: artifacts]
[2025-03-11 10:54:53,201: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-11 10:54:58,153: INFO: 4161348853: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c34feeb7eb837492d105827f2f819afcc14a90c51208eeaddc6dc25965e9afa8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C30D:38CBE0:B93FC:20C5B7:67CFC91C
Accept-Ranges: bytes
Date: Tue, 11 Mar 2025 05:24:45 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10243-MAA
X-Cache: M